In [1]:
# contains classes for the RRT algorithm
import random
import numpy
import math
import pygame


pygame 2.0.1 (SDL 2.0.14, Python 3.7.9)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
class RRTMap():
    
    def __init__(self, start, goal, MapDimensions, obsdim, obsnum):
        
        self.start = start #start location
        self.goal = goal # goal location
        self.MapDimensions = MapDimensions # dimensions of map
        self.Maph, self.Mapw = self.MapDimensions # split dimensions into height and width
        
        # display window settings
        self.MapWindowName = 'RRT Path Planning' # name of the window
        pygame.display.set_caption(self.MapWindowName) # display name of window on window
        
        self.map = pygame.display.set_mode((self.Mapw,self.Maph)) # set dimensions of the window
        self.map.fill((255,255,255)) # set map background to white
        self.nodeRad = 2 #radius of node
        self.nodeThickness = 0 # node thickness
        self.edgeThickness = 1 # thickness of the edge 
        
        self.obstacles =[] # obstacles
        self.obsdim = obsdim # dimesnion of obstacles
        self.obsnum = obsnum # number of obstacles
        
        # COLORS
        self.grey = (70,70,70)
        self.Blue = (0,0,255)
        self.Green = (0,255,0)
        self.Red = (255,0,0)
        self.Orange = (255,75,75)
        self.white = (255,255,255)

        
        
    def drawMap(self,obstacles):
        pygame.draw.circle(self.map,self.Green,self.start, self.nodeRad+5, 0) # 0 for a solid colored circle
        pygame.draw.circle(self.map,self.Orange,self.goal, self.nodeRad+20, 1) # 1 for an empty circle circle
    
        self.drawObs(obstacles) 
        
        
    def drawPath(self):
        pass
    
    
    def drawObs(self, obstacles):
        obstaclesList = obstacles.copy()
        
        while (len(obstaclesList)>0 ):
            obstacle = obstaclesList.pop(0) # pop the tope obstacle fromour list
            pygame.draw.rect(self.map,self.grey,obstacle) # draw obstacle onto map, as color grey
            
            
################################################################################################################
    
class RRTGraph():
    
    def __init__(self,start, goal, MapDimensions, obsdim, obsnum):
        
        (x,y) = start
        
        self.start = start # start location
        self.goal = goal # goal location
        self.goalFlag = False # goal flag is false (not reached)
        
        self.maph, self.mapw = MapDimensions # map deimensions 
        self.x =[] # x value of nodes
        self.y =[] # y value of nodes
        self.parent =[] # parent 
        
        # initialize the tree
        self.x.append(x) # add the x coord of start node to x
        self.y.append(y) # add the y coord of start node to y
        self.parent.append(0)  # root is 0
        
        # set up obstacle
        self.obstacles = []
        self.obsdim = obsdim
        self.obsnum = obsnum
        
        # path 
        self.goalstate = None # flag to indicate we have reached goal
        self.path = [] # store the path we take to goal
        
        
        
        
    
    def makeRandomRect(self):
        uppercornerx = int(random.uniform(0,self.mapw-self.obsdim)) # upper left corner x coordinate of a rectangle (cast as int)
        uppercornery = int(random.uniform(0,self.maph-self.obsdim)) # upper left corner y coordinate of a rectangle (cast as int)
        
        return(uppercornerx,uppercornery) # return duple of coordinates
    
    
    
    def makeObs(self):
        obs =[] # objects
        
        for i in range(0,self.obsnum):
            rectang = None # temporarily hold obstacle before it is stored (temp)
            start_goal_collision = True #check if goal or start is inside a newly created obstacle
            
            
            while start_goal_collision:
                upper = self.makeRandomRect()
                rectang =  pygame.Rect(upper,(self.obsdim,self.obsdim))  # use pygame to make a rectangle. Pass location of upper left
                                                                    # coordinate of triangle, and its dimensions
                
                # check if collision occurs withs tart or goal
                if rectang.collidepoint(self.start) or rectang.collidepoint(self.goal):
                    start_goal_collision = True # if collide, go back to while and try agian
                    
                else: 
                    start_goal_collision = False # break out of loop 
                    
            obs.append(rectang)
            
        self.obstacles = obs.copy() #copy obstacels to class 
        return obs # rturn list of obstacles 
    
    
                
                
    
    def add_node(self, n, x, y):
        self.x.insert(n,x)
        self.y.append(y)
        
    
    def remove_node(self,n):
        self.x.pop(n)
        self.y.pop(n)
        
    
    def add_edge(self, parent, child):
        self.parent.insert(child, parent)
    
    def remove_edge(self,n):
        self.parent.pop(n)
    
    def number_of_nodes(self):
        return len (self.x)
    
    def distance(self, n1,n2):
        (x1,y1) = (self.x[n1],self.y[n1])
        (x2,y2) = (self.x[n2],self.y[n2])
        px = (float(x1)-float(x2))**2
        py = (float(y1) - float(y2))**2
        
        return (px+py)**(0.5)
    
    
    def sample_env(self):
        x = int(random.uniform(0,self.mapw))
        y = int(random.uniform(0,self.maph))

        return x,y
    
    def nearest(self,n):
        dmin = self.distance(0,n)
        nnear = 0
        
        for i in range(0,n):
            if self.distance(i,n) <dmin:
                dmin = self.distance(i,n)
                nnear=i
        return nnear
    
    
    def isFree(self):
        n = self.number_of_nodes() - 1
        (x,y) = (self.x[n], self.y[n])
        obs = self.obstacles.copy()
        
        while len(obs) >0:
            
            rectang = obs.pop(0)
            
            if rectang.collidepoint(x,y):
                self.remove_node(n)
                return False
        return True
    
    
    def crossObstacle(self,x1,x2,y1,y2):
        obs = self.obstacles.copy()
        
        while(len(obs)>0):
            rectang = obs.pop(0)
            for i in range(0,101):
                u = i/100
                x = x1*u + x2*(1-u)
                y = y1*u + y2*(1-u)
                
                if rectang.collidepoint(x,y):
                    return True
        return False
    
    
    def connect(self, n1, n2):
        (x1,y1) = (self.x[n1],self.y[n1])
        (x2,y2) = (self.x[n2],self.y[n2])
        
        if self.crossObstacle(x1,x2,y1,y2):
            self.remove_node(n2)
            return False
        else:
            self.add_edge(n1,n2)
            return True
    
    def step(self,nnear,nrand,dmax=35):
        d = self.distance(nnear,nrand)
        if d>dmax:
            u = dmax/d
            (xnear,ynear) = (self.x[nnear], self.y[nnear])
            (xrand,yrand) = (self.x[nrand], self.y[nrand])
            (px,py) = (xrand-xnear, yrand-ynear)
            theta = math.atan2(py,px)
            
            (x,y) = (int(xnear+dmax*math.cos(theta)), int(ynear+dmax*math.sin(theta)))
            
            self.remove_node(nrand)
            
            if abs(x-self.goal[0]) <dmax and abs(y-self.goal[1]) <dmax:
                self.add_node(nrand,self.goal[0],self.goal[1])
                self.goalstate = nrand
                self.goalFlag = True
                
            else:
                self.add_node(nrand,x,y)
            
    
    def path_to_goal(self):
        pass
    
    def getPathCoords(self):
        pass
    
    def bias(self,ngoal):
        n = self.number_of_nodes()
        self.add_node(n,ngoal[0], ngoal[1])
        nnear = self.nearest(n)
        self.step(nnear,n)
        self.connect(nnear,n)
        return self.x,self.y,self.parent
    
    
    def expand(self):
        n = self.number_of_nodes()
        x,y = self.sample_env()
        self.add_node(n,x,y)
        
        if self.isFree():
            xnearest = self.nearest(n)
            self.step(xnearest,n)
            self.connect(xnearest,n)
        return self.x, self.y, self.parent
    
    def cost(self):
        pass
    
    
    